In [13]:
import getpass
import os
from pathlib import Path
from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()

In [14]:
from agno.models.huggingface import HuggingFace
from agno.embedder.ollama import OllamaEmbedder
from agno.models.ollama import Ollama
from agno.models.huggingface import HuggingFace
from agno.models.groq import Groq
from agno.agent import Agent
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.csv_toolkit import CsvTools


* LLM

In [15]:
hf_model=HuggingFace(
        id="meta-llama/Meta-Llama-3-8B-Instruct",
        api_key=os.getenv("HUGGINGFACE_API_TOKEN"),
)


ollama_model=Ollama(id="llama3.2:3b")

groq_model=Groq(id="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API_KEY"))

In [16]:
ollama_embedder = OllamaEmbedder(id="nomic-embed-text:latest",dimensions=768)

### Web Searcher Agent

In [50]:
web_searcher = Agent(model=groq_model, 
                     tools=[DuckDuckGoTools()], 
                     name="Web Searcher",  
                     role="Searches the web for information on a topic", 
                     show_tool_calls=True,
                     markdown=False)

response = web_searcher.run("Who is Thomas Edison?")
print(response.content) #latencia 1min 50

 - Running: duckduckgo_search(query=Thomas Edison biographylife, max_results=5)

It seems that Thomas Edison was an American inventor and businessman who developed many devices in fields such as electric power generation, mass communication, sound recording, and motion pictures. He is best known for his inventions of the phonograph, the motion picture camera, and early versions of the electric light bulb. Edison took out 1,093 patents in a variety of fields, including electric light and power, telephony and motion pictures. He was homeschooled by his mother due to being considered too difficult as a child.


### Csv Agent

In [68]:
csv_agent = Agent(
    model=groq_model,
    name="Csv Agent",
    role="Response to questions about employees csv file",
    tools=[CsvTools(csvs=["../data/employees.csv"])],
    markdown=False,
    show_tool_calls=True,
    debug_mode=False,
    instructions=[
        "First always get the list of files",
        "Then check the columns in the file",
        "Then run the query to answer the question",
        "Always wrap column names with double quotes if they contain spaces or special characters",
        "Remember to escape the quotes in the JSON string (use \")",
        "Use single quotes for string values"
    ],
)



In [73]:
response = csv_agent.run("What is the age of the  person with the column FirstName equal to 'John'  and the column LastName  equal to 'Doe' ?")
print(response.content)

INFO     Reading columns from file: employees

INFO     Loading csv file: employees

INFO     Running query: SELECT MAX(Age) FROM "employees" WHERE "FirstName" = 'John' AND "LastName" = 'Doe'

Running:
 - list_csv_files()
 - get_columns(csv_name=employees)
 - query_csv_file(csv_name=employees, sql_query=SELECT MAX(Age) FROM "employees" WHERE "FirstName" = 'John' AND "LastName" = 'Doe')

The maximum age of any person with the first name 'John' and the last name 'Doe' in the employees csv file is 28.


In [71]:
response = csv_agent.run("What is the Department of the  person with the column FirstName equal to 'John'  and the column LastName  equal to 'Doe' ?")
print(response.content)

INFO     Reading columns from file: employees

INFO     Loading csv file: employees

INFO     Running query: SELECT "Department" FROM employees WHERE "FirstName" = 'John' AND "LastName" = 'Doe'

Running:
 - list_csv_files()
 - get_columns(csv_name=employees)
 - query_csv_file(csv_name=employees, sql_query=SELECT "Department" FROM employees WHERE "FirstName" = 'John' AND "LastName" = 'Doe')

So the department of the person with the column FirstName equal to 'John'  and the column LastName  equal to 'Doe' is 'Engineering'.


### Multi-Agent Team

In [74]:
team_leader = Agent(
    name="Multi-Agent Team",
    role="Team leader agent",
    model=groq_model,
    tools=[web_searcher,csv_agent],
    show_tool_calls=True,
    markdown=False,
    debug_mode=False,
)


In [82]:
team_leader.run("Who is Thomas Edison?")
response.content

"To extract the Department of the person with the column 'FirstName' equal to 'John' and the column 'LastName' equal to 'Doe', I'll need to query a database. \n\nFor simplicity, let's assume we are working with a SQL database, and the table is named 'Employees'.\n\n```sql\nCREATE TABLE Employees (\n  Id INT PRIMARY KEY,\n  FirstName NVARCHAR(50),\n  LastName NVARCHAR(50),\n  Department NVARCHAR(100)\n);\n```\n\nWe can query this table to find the Department of the person with the column 'FirstName' equal to 'John' and the column 'LastName' equal to 'Doe' as follows:\n\n```sql\nSELECT Department \nFROM Employees \nWHERE FirstName = 'John' AND LastName = 'Doe';\n```\n\nThis SQL query will return the Department of the person matching the specified criteria.\n\nHowever, if multiple employees have the same name, the above query will return all their departments. If you are looking for a single department (e.g., the first or any department), we can use the LIMIT keyword (in MySQL) or FETCH F

In [81]:
response = team_leader.run("What is the Department of the  person with the column FirstName equal to 'John'  and the column LastName  equal to 'Doe' ?")
response.content

"To extract the Department of the person with the column 'FirstName' equal to 'John' and the column 'LastName' equal to 'Doe', I'll need to query a database. \n\nFor simplicity, let's assume we are working with a SQL database, and the table is named 'Employees'.\n\n```sql\nCREATE TABLE Employees (\n  Id INT PRIMARY KEY,\n  FirstName NVARCHAR(50),\n  LastName NVARCHAR(50),\n  Department NVARCHAR(100)\n);\n```\n\nWe can query this table to find the Department of the person with the column 'FirstName' equal to 'John' and the column 'LastName' equal to 'Doe' as follows:\n\n```sql\nSELECT Department \nFROM Employees \nWHERE FirstName = 'John' AND LastName = 'Doe';\n```\n\nThis SQL query will return the Department of the person matching the specified criteria.\n\nHowever, if multiple employees have the same name, the above query will return all their departments. If you are looking for a single department (e.g., the first or any department), we can use the LIMIT keyword (in MySQL) or FETCH F